# 1. Import Dependencies

In [58]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go

Fetch the data from the **COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University**

In [59]:
cases= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [60]:
deaths= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

# 2. Do Preprocessing on the Cases Data

In [61]:
cases.shape

(274, 457)

In [62]:
cases

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,57019,57144,57160,57242,57364,57492,57534,57612,57721,57793
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,127795,128155,128393,128518,128752,128959,129128,129307,129456,129594
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,118251,118378,118516,118645,118799,118975,119142,119323,119486,119642
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,12456,12497,12545,12581,12614,12641,12641,12712,12771,12805
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,23242,23331,23457,23549,23697,23841,23951,24122,24300,24389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,...,2683,2692,2693,2705,2714,2733,2758,2772,2781,2785
270,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,264395,265897,268132,270856,272767,274690,276407,278135,279753,280741
271,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,5233,5276,5357,5446,5507,5582,5657,5715,5770,5812
272,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,89783,89918,90029,90064,90218,90389,90532,90750,90844,90918


In [63]:
countries=cases["Country/Region"].count()
unique_countries=cases["Country/Region"].nunique(dropna = True)
print("There are ",countries ,"entries for the country column while unique no of countries is ",unique_countries)

There are  274 entries for the country column while unique no of countries is  192


This means there are repeated countries due to the seperate divisions for some countries inthe Province/State column like the US and Australia 

In [64]:
cases["Province/State"].nunique(dropna = True)

85

##### Drop the Province/State column, then group all rows by country.

In [65]:
# df1 shall be used as the dataframe to do preprocessing on
df1=cases.drop('Province/State',axis=1)
df1.shape

(274, 456)

In [66]:
df1["Country/Region"].nunique(dropna = True)

192

##### We have to obtain a similar number of rows to the one in the above output after grouping by country/Region

In [67]:
df1=df1.groupby(['Country/Region'],as_index=False).sum()
df1.shape

(192, 456)

##### Great, they match up, the groupby operation was successful

In [68]:
df1

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21
0,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,0,...,57019,57144,57160,57242,57364,57492,57534,57612,57721,57793
1,Albania,41.153300,20.168300,0,0,0,0,0,0,0,...,127795,128155,128393,128518,128752,128959,129128,129307,129456,129594
2,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,...,118251,118378,118516,118645,118799,118975,119142,119323,119486,119642
3,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,...,12456,12497,12545,12581,12614,12641,12641,12712,12771,12805
4,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,...,23242,23331,23457,23549,23697,23841,23951,24122,24300,24389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Vietnam,14.058324,108.277199,0,2,2,2,2,2,2,...,2683,2692,2693,2705,2714,2733,2758,2772,2781,2785
188,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,0,...,264395,265897,268132,270856,272767,274690,276407,278135,279753,280741
189,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,...,5233,5276,5357,5446,5507,5582,5657,5715,5770,5812
190,Zambia,-13.133897,27.849332,0,0,0,0,0,0,0,...,89783,89918,90029,90064,90218,90389,90532,90750,90844,90918


##### For the project, we don't need the Latitude and Longitude columns of our data.

In [69]:
df1=df1.drop(['Lat','Long'],axis=1)    

##### Obtain the Country codes from the Plotly Express documentation.Theyre useful in plotting the choropleth map

In [70]:
# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
# df

##### Drop the GDP column since we do not need it.

In [71]:
# df=df.drop('GDP (BILLIONS)',axis=1)
# df

In [72]:
df1

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,57019,57144,57160,57242,57364,57492,57534,57612,57721,57793
1,Albania,0,0,0,0,0,0,0,0,0,...,127795,128155,128393,128518,128752,128959,129128,129307,129456,129594
2,Algeria,0,0,0,0,0,0,0,0,0,...,118251,118378,118516,118645,118799,118975,119142,119323,119486,119642
3,Andorra,0,0,0,0,0,0,0,0,0,...,12456,12497,12545,12581,12614,12641,12641,12712,12771,12805
4,Angola,0,0,0,0,0,0,0,0,0,...,23242,23331,23457,23549,23697,23841,23951,24122,24300,24389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Vietnam,0,2,2,2,2,2,2,2,2,...,2683,2692,2693,2705,2714,2733,2758,2772,2781,2785
188,West Bank and Gaza,0,0,0,0,0,0,0,0,0,...,264395,265897,268132,270856,272767,274690,276407,278135,279753,280741
189,Yemen,0,0,0,0,0,0,0,0,0,...,5233,5276,5357,5446,5507,5582,5657,5715,5770,5812
190,Zambia,0,0,0,0,0,0,0,0,0,...,89783,89918,90029,90064,90218,90389,90532,90750,90844,90918


In [73]:
# Store this dataframe for later use
df1.to_excel ("cases.xlsx", index = False, header=True)

##### Drop all columns from the first date to the second last.
Since we need only the total cases on the last day for our map.

In [74]:
cols=df1[df1.columns[1:-1]]
df2=df1.drop(cols,axis=1)

In [75]:
#df2 will be the new sliced dataframe 
df2

,Country/Region,4/18/21
0,Afghanistan,57793
1,Albania,129594
2,Algeria,119642
3,Andorra,12805
4,Angola,24389
...,...,...
187,Vietnam,2785
188,West Bank and Gaza,280741
189,Yemen,5812
190,Zambia,90918


##### DO a left join on the Country/Region and COUNTRY columns

In [76]:
# df3=pd.merge(df2,df,how='left',left_on=['Country/Region'],right_on=['COUNTRY'])
# df3

##### Drop the COUNTRY column from the new dataframe (df3)

In [77]:
# df3=df3.drop('COUNTRY',axis=1)
# df3

##### Check whether there are any missing values due to mismatching Country names in the two datasets and missing codes 
##### for some countries in the left dataset.

In [78]:
# df3['CODE'].isnull().values.any()

In [79]:
# empty= df3[df3.isna().any(axis=1)]
# empty

##### Save the plotly map codes to a csv, edit it with all these missing codes, re-read it and do an inner join using the final version.

In [80]:
# df.to_excel ("map_code_copy.xlsx", index = False, header=True)

In [81]:
import os
pwd=os.getcwd()
df4=pd.read_excel(pwd+"\\map_code_copy.xlsx")

In [82]:
df4

,COUNTRY,CODE
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND
...,...,...
217,Virgin Islands,VGB
218,West Bank and Gaza,WBG
219,Yemen,YEM
220,Zambia,ZMB


##### Save it as a new file for use in final script

In [83]:
df4.to_excel ("map_code1.xlsx", index = False, header=True)

##### Perform inner join on the map code and cases(df2)
This step eliminates countries (Holy See,North Macedonia and Micronesia) and cruise ships (Diamond Princess and MS Zaandam)in the cases dataset.

In [84]:
df5=pd.merge(df2,df4,how='inner',left_on=['Country/Region'],right_on=['COUNTRY'])
df5=df5.drop('COUNTRY',axis=1)
df5

,Country/Region,4/18/21,CODE
0,Afghanistan,57793,AFG
1,Albania,129594,ALB
2,Algeria,119642,DZA
3,Andorra,12805,AND
4,Angola,24389,AGO
...,...,...,...
182,Vietnam,2785,VNM
183,West Bank and Gaza,280741,WBG
184,Yemen,5812,YEM
185,Zambia,90918,ZMB


In [85]:
# Check for missing values
empty= df5[df5.isna().any(axis=1)]
empty

,Country/Region,4/18/21,CODE


In [86]:
# Rename the Cases column
casemapdata = df5.rename(columns={df5.columns[1]: 'Total Cases'})
casemapdata

,Country/Region,Total Cases,CODE
0,Afghanistan,57793,AFG
1,Albania,129594,ALB
2,Algeria,119642,DZA
3,Andorra,12805,AND
4,Angola,24389,AGO
...,...,...,...
182,Vietnam,2785,VNM
183,West Bank and Gaza,280741,WBG
184,Yemen,5812,YEM
185,Zambia,90918,ZMB


# 3. Fit the data to the Plotly Choropleth map

In [87]:

fig = go.Figure(data=go.Choropleth(
    locations = casemapdata['CODE'],
    z = casemapdata['Total Cases'],
    text = casemapdata['Country/Region'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Total Cases',
))

fig.update_layout(
    title_text='Cumulative Cases per Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://github.com/CSSEGISandData/COVID-19">\
            JHU CSSE COVID-19 Data</a>',
        showarrow = False
    )]
)

fig.show()

In [88]:
casemapdata.to_excel ("casemapdata.xlsx", index = False, header=True)

# 4. Do Preprocessing on the Deaths Data

In [89]:
deaths.shape

(274, 457)

##### Drop the Province/State column, then group all rows by country.
##### df1 shall be used as the dataframe to do preprocessing on

In [90]:
df6=deaths.drop('Province/State',axis=1)
df6.shape

(274, 456)

In [91]:
df6["Country/Region"].nunique(dropna = True)

192

###### We have to obtain a similar number of rows to the one in the above output after grouping by country/Region

In [92]:
df6=df6.groupby(['Country/Region'],as_index=False).sum()
df6.shape

(192, 456)

##### Great, they match up, the groupby operation was successful

##### For the project, we don't need the Latitude and Longitude columns of our data.   

In [93]:
df6=df6.drop(['Lat','Long'],axis=1)    

In [94]:
# Store this dataframe for later use
df6.to_excel ("deaths.xlsx", index = False, header=True)

In [95]:
cols2=df6[df6.columns[1:-1]]
df7=df6.drop(cols2,axis=1)
#df7 will be the new sliced dataframe 
df7

,Country/Region,4/18/21
0,Afghanistan,2539
1,Albania,2342
2,Algeria,3155
3,Andorra,123
4,Angola,561
...,...,...
187,Vietnam,35
188,West Bank and Gaza,3017
189,Yemen,1126
190,Zambia,1235


In [96]:
df8=pd.merge(df7,df4,how='inner',left_on=['Country/Region'],right_on=['COUNTRY'])
df8=df8.drop('COUNTRY',axis=1)
df8

,Country/Region,4/18/21,CODE
0,Afghanistan,2539,AFG
1,Albania,2342,ALB
2,Algeria,3155,DZA
3,Andorra,123,AND
4,Angola,561,AGO
...,...,...,...
182,Vietnam,35,VNM
183,West Bank and Gaza,3017,WBG
184,Yemen,1126,YEM
185,Zambia,1235,ZMB


In [97]:
# Check for missing values
empty= df8[df8.isna().any(axis=1)]
empty

,Country/Region,4/18/21,CODE


In [98]:
# Rename the Cases column
deathmapdata = df8.rename(columns={df8.columns[1]: 'Total Deaths'})
deathmapdata

,Country/Region,Total Deaths,CODE
0,Afghanistan,2539,AFG
1,Albania,2342,ALB
2,Algeria,3155,DZA
3,Andorra,123,AND
4,Angola,561,AGO
...,...,...,...
182,Vietnam,35,VNM
183,West Bank and Gaza,3017,WBG
184,Yemen,1126,YEM
185,Zambia,1235,ZMB


# 5. Fit the data to the Plotly Choropleth map

In [99]:
fig = go.Figure(data=go.Choropleth(
    locations = casemapdata['CODE'],
    z = deathmapdata['Total Deaths'],
    text = casemapdata['Country/Region'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Total Deaths',
))

fig.update_layout(
    title_text='Cumulative Deaths per Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://github.com/CSSEGISandData/COVID-19">\
            JHU CSSE COVID-19 Data</a>',
        showarrow = False
    )]
)

fig.show()

In [100]:
deathmapdata.to_excel ("deathmapdata.xlsx", index = False, header=True)

In [101]:
df1=df1.transpose()

In [102]:
#dff1 = df1.loc[df1['Country/Region'] == 'Angola']
#dff1

In [103]:
#dff1.columns[1:]
df1

,0,1,2,3,4,5,6,7,8,9,...,182,183,184,185,186,187,188,189,190,191
Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
1/22/20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1/23/20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
1/24/20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
1/25/20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4/14/21,57492,128959,118975,12641,23841,1201,2604157,205128,29469,584205,...,4393330,152089,85503,3,178094,2733,274690,5582,90389,37369
4/15/21,57534,129128,119142,12641,23951,1209,2629156,206142,29484,586883,...,4396096,156499,85730,3,179365,2758,276407,5657,90532,37422
4/16/21,57612,129307,119323,12712,24122,1213,2658628,207103,29499,589299,...,4398903,159569,86022,3,180609,2772,278135,5715,90750,37534
4/17/21,57721,129456,119486,12771,24300,1216,2677747,207973,29519,591347,...,4401176,162400,86338,3,181903,2781,279753,5770,90844,37699


In [104]:
new_header = df1.iloc[0] #grab the first row for the header
df1 = df1[1:] #take the data less the header row
df1.columns = new_header #set the header row as the df header
df1

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
1/22/20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1/23/20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
1/24/20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
1/25/20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
1/26/20,0,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4/14/21,57492,128959,118975,12641,23841,1201,2604157,205128,29469,584205,...,4393330,152089,85503,3,178094,2733,274690,5582,90389,37369
4/15/21,57534,129128,119142,12641,23951,1209,2629156,206142,29484,586883,...,4396096,156499,85730,3,179365,2758,276407,5657,90532,37422
4/16/21,57612,129307,119323,12712,24122,1213,2658628,207103,29499,589299,...,4398903,159569,86022,3,180609,2772,278135,5715,90750,37534
4/17/21,57721,129456,119486,12771,24300,1216,2677747,207973,29519,591347,...,4401176,162400,86338,3,181903,2781,279753,5770,90844,37699


In [105]:
df1.columns

Index(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria',
       ...
       'United Kingdom', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela',
       'Vietnam', 'West Bank and Gaza', 'Yemen', 'Zambia', 'Zimbabwe'],
      dtype='object', name='Country/Region', length=192)

In [106]:
df1=df1.reset_index()
df1

Country/Region,index,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
0,1/22/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1/23/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
2,1/24/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
3,1/25/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
4,1/26/20,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,4/14/21,57492,128959,118975,12641,23841,1201,2604157,205128,29469,...,4393330,152089,85503,3,178094,2733,274690,5582,90389,37369
449,4/15/21,57534,129128,119142,12641,23951,1209,2629156,206142,29484,...,4396096,156499,85730,3,179365,2758,276407,5657,90532,37422
450,4/16/21,57612,129307,119323,12712,24122,1213,2658628,207103,29499,...,4398903,159569,86022,3,180609,2772,278135,5715,90750,37534
451,4/17/21,57721,129456,119486,12771,24300,1216,2677747,207973,29519,...,4401176,162400,86338,3,181903,2781,279753,5770,90844,37699


In [107]:
df1=df1.rename(columns = {'Country/Region':'index','index':'Date'})
df1

Country/Region,Date,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
0,1/22/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1/23/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
2,1/24/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
3,1/25/20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
4,1/26/20,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,4/14/21,57492,128959,118975,12641,23841,1201,2604157,205128,29469,...,4393330,152089,85503,3,178094,2733,274690,5582,90389,37369
449,4/15/21,57534,129128,119142,12641,23951,1209,2629156,206142,29484,...,4396096,156499,85730,3,179365,2758,276407,5657,90532,37422
450,4/16/21,57612,129307,119323,12712,24122,1213,2658628,207103,29499,...,4398903,159569,86022,3,180609,2772,278135,5715,90750,37534
451,4/17/21,57721,129456,119486,12771,24300,1216,2677747,207973,29519,...,4401176,162400,86338,3,181903,2781,279753,5770,90844,37699


In [108]:
df1.columns

Index(['Date', 'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       ...
       'United Kingdom', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela',
       'Vietnam', 'West Bank and Gaza', 'Yemen', 'Zambia', 'Zimbabwe'],
      dtype='object', name='Country/Region', length=193)

In [111]:
df1['Kenya'].dtypes

dtype('O')

In [49]:
#dff1.columns = range(dff1.shape[1])
#dff2=pd.Index(dff1.iloc[0])

In [50]:
#dff2[1:]

In [51]:
#dff1 = df1.loc[df1['Country/Region'] == 'Angola']

#dff1.columns = range(dff1.shape[1])
#dff2=pd.Index(dff1.iloc[0])

#fig = px.line(dff1, x=dff1.columns[1:], y=dff2[1:])    
#fig.show()


In [52]:
dfx = px.data.stocks()
dfx

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
...,...,...,...,...,...,...,...
100,2019-12-02,1.216280,1.546914,1.425061,1.075997,1.463641,1.720717
101,2019-12-09,1.222821,1.572286,1.432660,1.038855,1.421496,1.752239
102,2019-12-16,1.224418,1.596800,1.453455,1.104094,1.604362,1.784896
103,2019-12-23,1.226504,1.656000,1.521226,1.113728,1.567170,1.802472
